In [ ]:
# 1. 라이브러리 임포트
# ==============================================================================
import pvlib#태양광 발전 전문 태양 위치 등 근데 위동경도가 아니라 과연 하는게 더 나을지 의문
import pandas as pd
import numpy as np#원활한 계산을 위해 전체적으로 적용됨 pands lgb둘다
import gc#쓸데없는 메모리 회수처리
import os#윈도우와 상호작용(중복된거있으면 건너뛸수있게)

# 2. 설정 변수 정의
# ==============================================================================
# 파일 경로, 청크 크기 등 주요 설정을 상단에 정의하여 관리 용이성을 높입니다.
RAW_FILE_PATH = 'train.csv'#원본 데이터 파일 경로(변수통해서 지정)
CHUNK_SIZE = 5000000  # 메모리 상황에 맞춰 조절 (예: 50만 행)일단 만으로
TARGET = 'nins'
# 모델 학습에 사용할 최종 특성 리스트
FEATURES = [#'ground_press', 'wind_gust_spd', 'wind_chill_temp', 'rel_hum', #'type', 'nins', #energe 까지 7개 제외    26개
    'appr_temp', 'ceiling', 'cloud_b', 'dew_point', 'precip_1h', 'pressure',
     'temp_b', 'uv_idx', 'vis', 'wind_dir_b', 'wind_spd_b',
    'cloud_a', 'humidity', 'rain', 'snow', 'temp_a', 'wind_dir_a', 'wind_spd_a',
    'coord1', 'coord2', 'time',
    'pv_id', # 발전소 ID를 특성으로 사용할 예정
    'real_feel_temp_shade', 'temp_max', 'temp_min', 'real_feel_temp'#밑거름이 될 칼럼들
]
# 3. 데이터 처리 및 피처 엔지니어링 함수

def process_data_chunk(chunk_df):
    # 'pv_id'를 메모리 효율적인 category 타입으로 변환
    chunk_df['pv_id'] = chunk_df['pv_id'].astype('category')#반복되는 발전소번호등 간단히 저장(ex:pv_id1=1)
    
    # 함수 내부에 추가 #메모리최적화
    for col in chunk_df.select_dtypes(include=['float64']).columns:#float64타입선택(colums는 선택된 열들 col로봔한)
        chunk_df[col] = chunk_df[col].astype('float32')#32로 변환해서 col(정밀도 소폭감소 차이별로)
    for col in chunk_df.select_dtypes(include=['int64']).columns:
        chunk_df[col] = chunk_df[col].astype('int32')#정수형도 마찬가지
#들어온 청크 싹 변환하고 밑에서 그변환값 이용해서 새로운 변수 생성

    # 날짜/시간으로 변환 (erros는 혹시모를 오류방지(아예비어있거나그런)->자료에 그런거 없는거 같긴한데 혹시모르니)
    chunk_df['time'] = pd.to_datetime(chunk_df['time'], errors='coerce')
    
    # 최종적으로 사용할 FEATURES 리스트에 있는 열들만 선택하여 반환
    final_features = [f for f in FEATURES if f in chunk_df.columns]
    return chunk_df[final_features + [TARGET]]#추가로 얘는 필요+1

# --=함수정의 완료
# 4. 대용량 데이터 로딩 및 전처리 실행
processed_chunks = []#빈 리스트 생성
# chunksize 옵션을 사용하여 파일을 청크 단위로 순회
for chunk in pd.read_csv(RAW_FILE_PATH, chunksize=CHUNK_SIZE, engine='python', sep=','):#오류줄이기위해 엔진과 구분자 명시적 지정
    processed_chunk = process_data_chunk(chunk)#아까정의한 함수 이용 가공한 값들 리스트에 저장
    processed_chunks.append(processed_chunk)#청크 계속 누적시켜서 합치기위한 사전 만남의 광장

df = pd.concat(processed_chunks, ignore_index=True)# 처리된 모든 청크를 하나의 데이터프레임으로 결합
del processed_chunks#필요없어진 리스트 삭제
gc.collect()#메모리 회수
print("기본 처리 및 통합 완료.")
#--------------------------------------------------------------------------------------------------------- 최적화 파일불러오기

# 5. 추가 피처 엔지니어링 
print("\n고급 피처 엔지니어링을 시작합니다...")
df.sort_values(by=['pv_id', 'time'], inplace=True)#혹시 모를 오류로 인해 동일한 열(Column)이 두 번 생성되는 것을 방지하는 **'방어용 코드
# 기존의 중복되거나 순서가 섞인 인덱스가 있다면 버리고, 0부터 시작하는 새로운 인덱스를 부여(마찬가지 방어용)
df.reset_index(drop=True, inplace=True)
df = df.loc[:, ~df.columns.duplicated(keep='first')]#중복된 칼럼 제거(앞에꺼 살리고 뒤에꺼 버림)

#--------1.위치기반--------------------------------------------------------------------------------------------------------
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
print("위치 좌표(coord)를 기반으로 발전소 군집(Cluster) 특성을 생성합니다...")
#발전소별 평균 일사량 추정 1년치 전체
pv_id_stats = df.groupby('pv_id')[['nins']].mean().reset_index() #1. 발전소와 일사량
pv_id_stats.rename(columns={'nins': 'avg_nins'}, inplace=True)
unique_locations = df[['pv_id', 'coord1', 'coord2']].drop_duplicates(subset=['pv_id'])# 2 발전소와 위치만 존재

# 3. ★(수정)★ 고유 위치에 '평균 nins' 정보 병합
unique_locations = pd.merge(unique_locations, pv_id_stats, on='pv_id', how='left') #1,2를 발전소 기준으로 합치는 코드

# 4. ★(수정)★ KMeans에 사용할 특징 리스트 정의 (위치 + 지형 결과)
cluster_features = ['coord1', 'coord2', 'avg_nins']
# (결측치가 있으면 0으로 채움)
unique_locations[cluster_features] = unique_locations[cluster_features].fillna(0)#일몰된거겠지

# 5. ★(신규)★ StandardScaler로 특징들의 스케일 맞추기
# (coord와 avg_nins의 단위(스케일)가 다르므로, 공평한 비교를 위해 필수)
print("StandardScaler로 coord와 avg_nins의 단위를 통일합니다...")
scaler = StandardScaler()
scaled_features = scaler.fit_transform(unique_locations[cluster_features])

# 6. KMeans 모델로 19개의 '스마트 군집' 생성 (★10개 -> 19개★)
print(f"KMeans(n_clusters=19)로 {len(unique_locations)}개의 발전소를 3차원 공간에서 그룹화합니다...")
kmeans = KMeans(n_clusters=19, random_state=42, n_init=10)
unique_locations['location_cluster'] = kmeans.fit_predict(scaled_features)
    

print("메모리 효율적인 .map()을 위해 'cluster_map'을 생성합니다...")
# (예: {100: 0, 101: 3, 102: 0, ...})
cluster_map = unique_locations.set_index('pv_id')['location_cluster']

# 4. ★(수정)★ .merge() 대신 .map()을 사용하여 열을 추가 (메모리 사용량 훨씬 적음)
print(".map()을 사용하여 'location_cluster' 열을 추가합니다...")
df['location_cluster'] = df['pv_id'].map(cluster_map)
df['location_cluster'] = df['location_cluster'].astype('category')
print("위치 군집 매핑 완료.")


#시간관련을 먼저하는게  클러스터별 칼럼 설정? 을 위해
df['minute'] = df['time'].dt.minute
df['hour'] = df['time'].dt.hour
df['decimal_hour'] = df['hour'] + (df['minute'] / 60.0)
df['day_of_year'] = df['time'].dt.dayofyear
df['hour_sin'] = np.sin(2 * np.pi * df['decimal_hour'] / 24.0)
df['hour_cos'] = np.cos(2 * np.pi * df['decimal_hour'] / 24.0)
df['day_of_year_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365.25)
df['day_of_year_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365.25)
df['seasonal_hour'] = df['day_of_year_sin'] * df['hour_sin']

#--------3. ★★★ '맵' 파일 2종 생성 및 'Anomaly' 특징 적용 ★★★ -----------------------------------

# ★★★ 3A. '일출/일몰 맵' 생성 (nins > 0 사용) ★★★
print("클러스터별 '개인화 일출/일몰 맵'을 생성합니다...")
daytime_df = df[df['nins'] > 0].copy()
cluster_times = daytime_df.groupby('location_cluster')['decimal_hour'].agg(['min', 'max']).reset_index()
cluster_times.rename(columns={'min': 'sunrise_decimal', 'max': 'sunset_decimal'}, inplace=True)
cluster_times['sunrise_decimal'] += (5/60) # 5분 마진
cluster_times['sunset_decimal'] -= (5/60)  # 5분 마진
cluster_times.to_csv('cluster_sun_times_map.csv', index=False)
print("일출/일몰 맵 저장 완료.")
# (생성된 맵 특징을 train 데이터에도 병합 - 학습에 사용)
df = pd.merge(df, cluster_times, on='location_cluster', how='left')

# ★★★ 3B. '개인화 기준선(Anomaly) 맵' 생성 ★★★
print("GroupBy 개인화 기준선 맵 생성 중...")
base_df = df.groupby(['location_cluster', 'hour'])[['cloud_a', 'uv_idx', 'temp_a','humidity']].mean().reset_index()
base_df.rename(columns={
    'cloud_a': 'cluster_hour_cloud_mean',
    'uv_idx': 'cluster_hour_uv_mean',
    'temp_a': 'cluster_hour_temp_mean',
    'humidity': 'cluster_hour_humidity_mean'
}, inplace=True)
base_df.to_csv('cluster_hour_means_map.csv', index=False)
print("개인화 기준선 맵 저장 완료.")

# ★★★ 3C. Anomaly 특징을 train 데이터에 적용 ★★★
print("'평소 대비 이상 기후' (Anomaly) 특징 생성 중...")
df = pd.merge(df, base_df, on=['location_cluster', 'hour'], how='left')
df['cloud_anomaly'] = df['cloud_a'] - df['cluster_hour_cloud_mean']
df['uv_anomaly'] = df['uv_idx'] - df['cluster_hour_uv_mean']
df['temp_anomaly'] = df['temp_a'] - df['cluster_hour_temp_mean']
df['humidity_anomaly'] = df['humidity'] - df['cluster_hour_humidity_mean'] 



# --- 2.weather 날씨 '추세' 특성 생성-------------------------------------------------------------------------------------- 
print("날씨 추세(Rolling) 특성 생성 중...")
g = df.groupby('pv_id')
#추세
df['uv_idx_rolling_12'] = g['uv_idx'].shift(1).rolling(12, min_periods=1).mean()
df['uv_idx_lag_12'] = g['uv_idx'].shift(12)
df['humidity_rolling_12'] = g['humidity'].shift(1).rolling(12, min_periods=1).mean()
df['humidity_lag_12'] = g['humidity'].shift(12)
df['pressure_rolling_12'] = g['pressure'].shift(1).rolling(12, min_periods=1).mean()
df['pressure_lag_12'] = g['pressure'].shift(12)#UV,습도기압추세추가 기존것과 다르게 과거값이라 상관없음
df['cloud_a_rolling_12'] = g['cloud_a'].shift(1).rolling(12, min_periods=1).mean()#1시간 전 구름량의 1시간 이동평균
df['temp_a_rolling_12'] = g['temp_a'].shift(1).rolling(12, min_periods=1).mean()#1시간 전 기온의 1시간 이동평균
df['cloud_a_lag_12'] = g['cloud_a'].shift(12) #1시간 전 구름량 존재이유? ex:1시간 전에는 매우 흐렸고 1시간 평균도 높았는데, 지금은 맑아졌다. 날씨가 급격히 좋아지고 있다."
df['temp_a_lag_12'] = g['temp_a'].shift(12) #1시간 전 기온 존재이유? ex:1시간 전에는 매우 더웠고 1시간 평균도 높았는데, 지금은 시원해졌다. 날씨가 급격히 좋아지고 있다."
# 변동성(std) 특성 추가
df['cloud_a_rolling_12_std'] = g['cloud_a'].shift(1).rolling(12, min_periods=1).std() #이건 표준편차
df['uv_idx_rolling_12_std'] = g['uv_idx'].shift(1).rolling(12, min_periods=1).std()
df['pressure_rolling_12_std'] = g['pressure'].shift(1).rolling(12, min_periods=1).std()
df['temp_a_rolling_12_std'] = g['temp_a'].shift(1).rolling(12, min_periods=1).std()

#차이
df['is_precipitating'] = ((df['precip_1h'] > 0) | (df['rain'] > 0) | (df['snow'] > 0)).astype(int)#강수여부
df['station_temp_diff'] = df['temp_b'] - df['temp_a']#기온차이
df['station_cloud_diff'] = df['cloud_a'] - df['cloud_b']
df['station_wind_spd_diff'] = df['wind_spd_a'] - df['wind_spd_b']



 
#--- 3. 물리 기반 및 순환 특성 생성 ---
print("물리 기반 및 순환 특성 생성 중...")

df['temp_a_sq'] = df['temp_a']**2
df['uv_idx_sq'] = df['uv_idx']**2#2계절관련, 너무덥거나 추울때 떨어지는 효율 반영
df['sun_exposure_factor'] = df['real_feel_temp'].values - df['real_feel_temp_shade'].values#포화부족temp_a, dew_point는 제거고려
df['diurnal_temp_range'] = df['temp_max'].values - df['temp_min'].values#일교차, temp_max, temp_min는 제거 고려
df['saturation_deficit'] = df['temp_a'].values - df['dew_point'].values # 햇빛노출효과, real_feel_temp, real_feel_temp_shade는 제거 고려
df['is_foggy'] = (df['vis'] < 1).astype(int) # 가시거리가 1km 미만이면 1 (안개)

# 풍향(Wind Direction)의 순환성 변환 
print("풍향(Wind Direction) 순환 특성 생성 중...")#wind_dir_a와 wind_dir_b는 제거 1도와 361도를 다르게 만들가능성있음
df['wind_dir_a_sin'] = np.sin(2 * np.pi * df['wind_dir_a'] / 360.0)
df['wind_dir_a_cos'] = np.cos(2 * np.pi * df['wind_dir_a'] / 360.0)
df['wind_dir_b_sin'] = np.sin(2 * np.pi * df['wind_dir_b'] / 360.0)
df['wind_dir_b_cos'] = np.cos(2 * np.pi * df['wind_dir_b'] / 360.0)
df['wind_power_a'] = df['wind_spd_a'] ** 3#풍향세기
df['wind_power_b'] = df['wind_spd_b'] ** 3
# 2. 방향별 바람의 세기 (예: "북/남풍"의 세기, "동/서풍"의 세기)
df['wind_force_a_cos'] = df['wind_spd_a'] * df['wind_dir_a_cos']
df['wind_force_a_sin'] = df['wind_spd_a'] * df['wind_dir_a_sin']
df['wind_force_b_cos'] = df['wind_spd_b'] * df['wind_dir_b_cos']
df['wind_force_b_sin'] = df['wind_spd_b'] * df['wind_dir_b_sin']

# 시간-날씨 상호작용 특성 추가
df['temp_x_hour'] = df['temp_a'] * df['hour_cos']
df['cloud_x_hour'] = df['cloud_a'] * df['hour_cos']
df['uv_idx_x_hour'] = df['uv_idx'] * df['hour_cos']
df['humidity_x_hour'] = df['humidity'] * df['hour_cos']#중복되는 정보는?
gc.collect()#메모리 회수
#-------------------------------------------------------------------------------------------------------------------------------

print("최종 결측치 처리 중...")

# 1. Lag/Rolling/Std 특성은 0으로 먼저 채웁니다., lag관련은 다 포함시킨거임 이게
#-------------------------------------------------------------------------------------------------------------------------------

print("최종 결측치 처리 중...")

# 1. Lag/Rolling/Std 특성은 0으로 먼저 채웁니다.
lag_rolling_cols = [col for col in df.columns if '_lag_' in col or '_rolling_' in col]
df[lag_rolling_cols] = df[lag_rolling_cols].fillna(0)

# 2. 0/1 플래그(Flag) 변수들은 0으로 먼저 채웁니다.
if 'is_precipitating' in df.columns:
     df['is_precipitating'] = df['is_precipitating'].fillna(0)
if 'is_foggy' in df.columns: 
     df['is_foggy'] = df['is_foggy'].fillna(0)

# 3. ★★★ (수정) '화이트리스트' 방식: 보간할 '원본 센서 값'만 명시 ★★★
# (가공된 특징(_sq, _sin, _anomaly 등)은 이 리스트에 없으므로, 보간(interpolate)되지 않습니다)
cols_for_interpolation = [
    'appr_temp', 'ceiling', 'cloud_b', 'dew_point', 'precip_1h', 'pressure',
    'temp_b', 'uv_idx', 'vis', 'wind_dir_b', 'wind_spd_b',
    'cloud_a', 'humidity', 'rain', 'snow', 'temp_a', 'wind_dir_a', 'wind_spd_a',
    'real_feel_temp_shade', 'temp_max', 'temp_min', 'real_feel_temp'
]
# (혹시 파일 생성 시 FEATURES에서 제외했을 경우를 대비해, df에 존재하는 열만 선택)
cols_for_interpolation = [col for col in cols_for_interpolation if col in df.columns]

# 4. '연속형' 특성에 대해서만 발전소별 보간법 실행 (데이터 오염 방지)
print(f"다음 {len(cols_for_interpolation)}개 열(순수 날씨 변수)에 대해 ★발전소별 선형★ 보간을 수행합니다.")
df[cols_for_interpolation] = df.groupby('pv_id')[cols_for_interpolation].transform(lambda x: x.interpolate(method='linear', limit_direction='both'))

# 5. 보간 후 남은 모든 NaN은 0으로 최종 처리 (안전장치)
# (보간 대상이 아니었던 _anomaly, _sin, _sq 등의 NaN 값들이 여기서 0으로 처리됨)
df.fillna(0, inplace=True)
gc.collect()

# 6. float32 변환 (메모리 최적화)
print("모든 float64 칼럼을 float32로 강제 변환하여 메모리 사용량을 절반으로 줄입니다...")
# (select_dtypes 대신, df.columns를 순회하며 float64를 찾습니다 - 32비트 오류 방지)
float64_cols = [col for col in df.columns if df[col].dtype == 'float64']
for col in float64_cols:
    df[col] = df[col].astype('float32')

print(f"{len(float64_cols)}개 칼럼을 float32로 변환 완료.")
gc.collect()
# =============================================================================
# Parquet 저장을 위해 pyarrow 라이브러리가 필요 (pip install pyarrow)#import할 필요는없음 백엔드엔진이라
print("중간 처리된 데이터를 'processed_train.feather' 파일로 저장합니다...")
df.to_feather('processed_train.feather')#10분에 통합 청크더 올릴까 이제 메모리도 충분한데
print("저장 완료.")#82r

KeyboardInterrupt: 

In [3]:
import pandas as pd
df = pd.read_feather('processed_train.feather')
print(df.shape)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 75)
pd.set_option('display.width', 1000)
print(df.head())##현재 위도경도 포함됨 6추가


(19236948, 83)
   appr_temp  ceiling    cloud_b  dew_point  precip_1h  pressure     temp_b  uv_idx        vis  wind_dir_b  wind_spd_b   cloud_a   humidity  rain  snow     temp_a  wind_dir_a  wind_spd_a    coord1    coord2                      time    pv_id  real_feel_temp_shade   temp_max  temp_min  real_feel_temp  nins location_cluster  minute  hour  decimal_hour  day_of_year  hour_sin  hour_cos  day_of_year_sin  day_of_year_cos  seasonal_hour  ...  humidity_anomaly  uv_idx_rolling_12  uv_idx_lag_12  humidity_rolling_12  humidity_lag_12  pressure_rolling_12  pressure_lag_12  cloud_a_rolling_12  temp_a_rolling_12  cloud_a_lag_12  temp_a_lag_12  cloud_a_rolling_12_std  uv_idx_rolling_12_std  pressure_rolling_12_std  temp_a_rolling_12_std  is_precipitating  station_temp_diff  temp_a_sq  uv_idx_sq  sun_exposure_factor  diurnal_temp_range  saturation_deficit  is_foggy  wind_dir_a_sin  wind_dir_a_cos  wind_dir_b_sin  wind_dir_b_cos  wind_power_a  wind_power_b  wind_force_a_cos  wind_force_a

In [9]:
import pyarrow as pa
import lightgbm as lgb# lgb명칭으로 lightgbm외부라이브워리 불러옴
import pvlib#태양광 발전 전문 태양 위치 등
import pandas as pd
import numpy as np
import dask as dd

print("pyarrow가 설치되었으며 버전은:", pa.__version__)
print("pvlib가 설치되었으며 버전은:", pvlib.__version__)
print("pandas가 설치되었으며 버전은:", pd.__version__)
print("lightgbm이 설치되었으며 버전은:", lgb.__version__)
print("numpy가 설치되었으며 버전은:", np.__version__)
print("dask가 설치되었으며 버전은:", dd.__version__)



pyarrow가 설치되었으며 버전은: 21.0.0
pvlib가 설치되었으며 버전은: 0.13.1
pandas가 설치되었으며 버전은: 2.3.2
lightgbm이 설치되었으며 버전은: 4.6.0
numpy가 설치되었으며 버전은: 2.3.3
dask가 설치되었으며 버전은: 2025.9.1
